<img align="right" style="max-width: 200px; height: auto" src="./sfao_logo.jpg">

##  Lab 03 - Agentic Audit Models

Januar Schulung, Eidgenössische Finanzkontrolle (EFK), 2025

Die Labs zur **Künstlichen Intelligenz** der Januar Schulung 2025 basieren auf Jupyter Notebook. Anhand solcher Notebooks ist es möglich eine Vielzahl von Datenanalysen und statistischen Validierungen durchzuführen.

<img align="center" style="max-width: 700px" src="./lab_03_banner.png">

In diesem Lab werden wir untersuchen, wie **Künstliche Intelligenz (KI)** genutzt werden kann, um Prüfungsaufgaben in einem agentenbasierten Umfeld umzusetzen und durchzuführen. Ziel ist es, mehrere KI-Agenten einzusetzen, die gemeinsam Buchhaltungsbuchungen analysieren und so die Arbeit menschlicher Prüfer simulieren. Dieser Ansatz basiert auf dem Konzept des [**Agentic Auditing**](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4909147), bei dem spezialisierte KI-Agenten unterschiedliche Aufgaben übernehmen, wie zum Beispiel Anomalieerkennung, Compliance-Prüfung und Analyse von Freigabeprozessen. Jeder Agent arbeitet autonom, agiert jedoch als Teil eines Teams, um sicherzustellen, dass Prüfungsprozesse gründlich und effizient ablaufen. Die folgende Abbildung veranschaulicht den **agentenbasierten Prüfungsprozess von Buchhaltungsbuchungen**, den wir in diesem Lab aufbauen möchten.

<img align="center" style="max-width: 1000px" src="./audit_process.png">

Wir werden mehrere Agenten mit dem [**CrewAI**](https://www.crewai.com/) (<img align="center" style="max-width: 60px" src="./crewai_logo.png">) Framework implementieren und ausführen, um gemeinsam Buchhaltungsbuchungen zu prüfen. `CrewAI` ist ein Open-Source-Framework, das Entwicklern ermöglicht, automatisierte Workflows zu erstellen und bereitzustellen, indem mehrere KI-Agenten zusammenarbeiten, um komplexe Aufgaben zu erfüllen. Das entsprechende `CrewAI` GitHub-Repository ist über den folgenden Link erreichbar: [https://github.com/crewAIInc/crewAI](https://github.com/crewAIInc/crewAI).

Mit dem `CrewAI` Framework werden wir ein Team von KI-Agenten (wie oben dargestellt) implementieren und bereitstellen, um Buchhaltungsbuchungen gemäß dem [International Standard on Auditing (ISA) 240](https://www.ifac.org/_flysystem/azure-private/publications/files/A012%202013%20IAASB%20Handbook%20ISA%20240.pdf) zu prüfen. Jeder Prüfungsagent wird spezifische Aufgaben übernehmen, wie (i) Dokumentenanalyse, (ii) Datenbankabfragen und (iii) Berichtserstellung. Nach der Einrichtung der Agenten lassen wir sie in einem sequentiellen Prozess zusammenarbeiten, um die Prüfung der Buchhaltungsbuchungen durchzuführen.

Bei etwaigen Fragen wenden Sie sich, wie immer gerne an uns via **marco (dot) schreyer (at) efk (dot) admin (dot) ch**. Wir wünschen Ihnen Viel Freude mit unseren Notebooks und Ihren revisorischen Analysen!

## Lernziele des Labs:

Nach dem heutigen Lab sollten Sie in der Lage sein:

> 1. **Agentic Auditing verstehen:** Die Grundlagen des Einsatzes von KI-Agenten für Prüfungsaufgaben erfassen.  
> 2. **KI-Agenten mit CrewAI implementieren:** Agenten einrichten und konfigurieren, um Prüfungsaufgaben durchzuführen.  
> 3. **Agenten für die Prüfung von Buchhaltungsbuchungen einsetzen:** Agenten verwenden, um Anomalien in Buchhaltungsbuchungen zu erkennen.  
> 4. **Agentenleistung bewerten:** Die Leistung der Agenten anhand prüfungsrelevanter Metriken analysieren.  
> 5. **Prüfungsergebnisse berichten:** Prüfungsergebnisse visualisieren und zusammenfassen.  

Am Ende des Labs werden Sie verstehen, wie man mit dem `CrewAI` Framework KI-Agenten einrichtet, ihnen spezifische Prüfungsaufgaben zuweist und sie gemeinsam die Prüfung von Buchhaltungsbuchungen durchführen lässt.


## 1. Einrichten der Analyseumgebung

Ähnlich wie in den vorangegangenen Übungen werden wir zunächst eine Reihe von Python-Bibliotheken importieren, welche die Datenanalyse und -visualisierung ermöglichen. In dieser Übung werden wir die Bibliotheken `CrewAI`, `Pandas`, `Numpy`, `Scikit-Learn`, `Matplotlib` und `Seaborn` verwenden. Nachfolgend importieren wir die benötigten Bibliotheken durch die Ausführung der folgenden Anweisungen:

In [3]:
# import python data science and utility libraries
import os, requests
from datetime import datetime
from IPython.display import Markdown, display
from textwrap import dedent

Import der `Pandas` Data Science Bibliotheken:

In [4]:
# import import python data science libraries
import pandas as pd
import tabulate as tb

Import der `SQLAlchemy` Datenbank Bibliotheken:

In [ ]:
# import SQLAlchemy components for database interactions
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Date, insert

Installation der `crewAI` Agenten Bibliotheken:

In [ ]:
# install the crewai library and tools
! pip -qqq install crewai==0.79.4
! pip -qqq install crewai_tools==0.14.0

Import der `crewAI` Agenten Bibliotheken:

In [ ]:
# import the crewai library and tools
from crewai import Crew, Agent, Process, Task, LLM
from crewai_tools import BaseTool

Installation der `langchain` Large Language Model (LLM) Chaining Bibliothek:

In [ ]:
! pip -qqq install langchain_openai 

Import der `langchain` Large Language Model (LLM) Chaining Bibliothek:

In [ ]:
# import langchain
from langchain_openai import ChatOpenAI

## 2. Einrichtung des Large Language Models (LLM)

In diesem Abschnitt richten wir zwei verschiedene Large Language Models (LLMs) ein, die unsere Agenten für prüfungsbezogene Aufgaben nutzen werden. Wir beginnen mit einem lokal gehosteten LLM, das über die [**Ollama**](https://ollama.com/) Anwendung bereitgestellt wird, und erkunden anschließend den Zugriff auf das remote gehostete [**ChatGPT**](https://platform.openai.com/docs/overview) über die API von OpenAI. Diese Einrichtung ermöglicht es uns, sowohl lokal als auch cloudbasierte LLMs für agentenbasierte Prüfungsaufgaben zu testen. In beiden Umgebungen verwenden wir die **Langchain**-Schnittstelle `ChatOpenAI`, um auf die Modelle zuzugreifen.

### 2.1 Einrichtung eines lokalen LLM mit Ollama

Die **Ollama**-Anwendung bietet die Möglichkeit, LLMs lokal auszuführen. Wir werden das **LLaMA3**-Modell von Meta.ai über **Ollama** einrichten und verwenden. Dies ermöglicht eine lokale Ausführung des Modells, reduziert die Latenz und verbessert den Datenschutz. Um dies zu erreichen, laden Sie zunächst die Anwendung über den folgenden Link für Ihr Betriebssystem herunter und installieren Sie sie: [https://ollama.com/download](https://ollama.com/download).

Nach erfolgreicher Installation führen wir den Befehl `ollama list` aus, um die verfügbaren Modelle anzuzeigen:

In [ ]:
! ollama list

Unser Ziel ist es, das **LLaMA 3.2**-Modell zu verwenden, das instruktionstauglich ist und über 3 Milliarden Parameter verfügt (Text-in/Text-out). Dieses Modell wurde für mehrsprachige Dialogfälle optimiert, einschließlich agentenbasierter Such- und Zusammenfassungsaufgaben. Wir laden das **LLaMA 3.2**-Modell in der `Google Colab`-Anwendung mit dem `ollama pull`-Befehl herunter:

In [ ]:
! ollama pull llama3.2:3b

Anschließend überprüfen wir kurz, ob das Modell erfolgreich heruntergeladen wurde:

In [ ]:
! ollama list

Perfekt! Jetzt führen wir das Modell lokal auf Ihrem Gerät mit dem Befehl `ollama run` und dem entsprechenden Modellnamen aus:

In [ ]:
! ollama run llama3.2:3b

Da das Modell jetzt läuft, verwenden wir die **Langchain**-Schnittstelle `ChatOpenAI`, um eine Verbindung zu dem lokal gehosteten **LLaMA 3.2**-Modell herzustellen:

In [ ]:
# initialize the local LLaMA 3.2 by meta.ai
llm_llama = LLM(model='ollama/llama3.2:3b', base_url='http://localhost:11434/', temperature=0.9, max_tokens=2048, seed=1234)

Als Nächstes testen wir das lokale **LLaMA3**-Modell. Wir simulieren ein Szenario, in dem die KI aufgefordert wird, einen Spesenbericht eines etwas eigenwilligen Mitarbeiters zu prüfen, der möglicherweise private Ausgaben einreichen möchte. Dieser Test stellt sicher, dass das Modell korrekt reagiert:

In [ ]:
# define a fun audit prompt 
prompt = """
Imagine you're an auditor reviewing an expense report from an employee who claims all expenses are legitimate business costs. 
Here's a list of items they've submitted:

1. Office supplies - $100
2. Coffee machine - $250
3. Unicorn onesie - $45
4. 10-pound gummy bear - $30
5. "Business trip" to Disneyland - $2000
6. Sushi for team meeting - $150

Please identify which items are legitimate business expenses and which might require further investigation, with a bit of humor in your response. 
"""

Das lokale **LLaMA3**-Modell wird beauftragt, zu bestimmen, welche Posten im Spesenbericht legitim sind und welche einer genaueren Prüfung bedürfen. Das Modell sollte eine humorvolle, aber praktische Analyse der Ausgaben liefern:

In [ ]:
# test the model with this fun audit prompt
print(llm_llama.call(messages=[{ "content": prompt, "role": "user"}]))

### 2.2 Einrichtung eines cloudbasierten LLM mit Langchain

Die **OpenAI API** ermöglicht den Zugriff auf Sprachmodelle wie **GPT-4** (ChatGPT) über die Cloud. In diesem Abschnitt richten wir ChatGPT ein, indem wir einen API-Schlüssel konfigurieren und eine Verbindung zu den Servern von OpenAI herstellen. Dieser Ansatz ist vorteilhaft für Nutzer, die Zugriff auf ein leistungsstarkes Modell benötigen, jedoch keine lokale Hosting-Möglichkeit haben.

Um zu beginnen, erstellen Sie ein OpenAI-Konto und erhalten Sie einen API-Schlüssel über das **API-Dashboard** von OpenAI: [https://platform.openai.com](https://platform.openai.com).

Nach Erhalt des API-Schlüssels richten wir ihn als Umgebungsvariable ein. Dadurch wird der Schlüssel geschützt und unser Code kann mit dem **GPT-4**-Modell interagieren:

In [ ]:
# set the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = 'your_actual_openai_api_key_here'  # Replace with your actual OpenAI API key

Nun verwenden wir die `ChatOpenAI`-Bibliothek, um das **GPT-4**-Modell über die OpenAI API zu initialisieren. Stellen Sie sicher, dass der API-Schlüssel korrekt eingerichtet ist, bevor Sie diesen Code ausführen.

In [ ]:
# initialize ChatGPT with OpenAI's GPT-4 model
llm_gpt = ChatOpenAI(model='gpt-4', api_key=os.getenv('OPENAI_API_KEY'), temperature=0.7)

Mit der erfolgreichen Verbindung zu **GPT-4** über die OpenAI API können wir dieses cloudbasierte LLM für unsere agentenbasierten Prüfungsaufgaben einsetzen. Diese Einrichtung bietet Zugang zu leistungsstarken Sprachfunktionen und nutzt gleichzeitig die Infrastruktur von OpenAI.

Als Nächstes testen wir das cloudbasierte **GPT-4**-Modell! Anstatt üblicher Prüfungsaufgaben stellen wir ihm eine philosophische Frage nach dem Sinn des Lebens. Diese spielerische Aufgabe zeigt, wie gut das Modell analytische Anfragen bewältigen kann – genau wie es ein guter Prüfer tun würde:

In [ ]:
# define a fun audit prompt
prompt = """
Imagine you're an auditor known for precision and rigor, and one day, a curious question lands on your desk: 

"What is the meaning of life?" 

Using your audit skills, approach this question as if you’re conducting an audit. What evidence might you seek, 
what factors would you evaluate, and what red flags could arise in your quest to give a well-substantiated answer?
Add a touch of humor in your response, because, after all, who better to weigh in on life’s mysteries than an auditor?
"""

Senden wir die Anfrage an das cloudbasierte **GPT-4**-Modell. Das Modell sollte eine humorvolle Antwort auf die Frage liefern:

In [ ]:
# test the model with this fun audit prompt
print(llm_gpt.invoke(prompt).content)

In diesem Abschnitt haben wir zwei leistungsstarke Möglichkeiten erkundet, Sprachmodelle in unser Prüfungstoolkit zu integrieren: die Einrichtung eines lokalen Modells mit **Ollama** und den Zugriff auf das **GPT-4**-Modell über den Cloud-Dienst von OpenAI. Diese Tools bieten Flexibilität bei der Durchführung agentenbasierter Prüfungsaufgaben und balancieren Datenschutz und Leistungsfähigkeit aus. Mit dieser eingerichteten Umgebung sind wir bereit, uns praktischen Anwendungen zuzuwenden, bei denen diese Modelle als Prüfungsagenten in einem definierten Prüfungsprozess eingesetzt werden.

## 3. Etablierung einer agentenbasierten Prüfung von Journal Entries

In diesem Abschnitt führen wir Sie durch die Erstellung der grundlegenden Agenten unseres Prüfungsteams. Wir beginnen mit der Einrichtung eines **Senior Auditor**-Agenten, der komplexe organisatorische Richtlinien analysieren und für Prüfungszwecke zusammenfassen kann. Anschließend erweitern wir das Team um zusätzliche Agenten, die gemeinsam im Rahmen eines agentenbasierten Workflows die Prüfung von Buchhaltungsbuchungen durchführen.

### 3.1 Einrichtung eines einzelnen Prüfungsagenten

Der **Senior Auditor** ist der erste Agent in unserem Prüfungsteam. Er wurde entwickelt, um komplexe Richtlinien zu analysieren und umsetzbare Erkenntnisse zu generieren. Seine Hauptaufgabe besteht darin, organisatorische Richtlinien zu lesen und prägnant für Prüfungszwecke zusammenzufassen. Das "Gehirn" dieses Agenten wird vom **LLaMA 3.2-Modell** betrieben, das wir zuvor eingerichtet haben und das ihm ermöglicht, sprachbasierte Aufgaben wie Textanalyse und -zusammenfassung zu bewältigen (siehe Abbildung 2). Lassen Sie uns die Schritte zur Erstellung und Aufgabenvergabe an diesen Agenten durchgehen.

<img align='center' style="max-width: 850px" src='./senior_auditor_agent.png'>

**Figure 2: Senior Auditor Agent - Tools and Capabilities**

Der **Senior Auditor** benötigt ein Werkzeug, um die organisatorischen Richtlinien zu lesen. Wir stellen diese Funktionalität bereit, indem wir das Tool `ReadGuidelines` definieren. Dieses Tool liest eine Textdatei mit den Richtlinien und gibt deren Inhalt zurück. Dadurch erhält der Agent Zugriff auf die Richtlinien und kann sie verarbeiten.

In [ ]:
# define the ReadGuidelines tool for the Senior Auditor agent
class ReadGuidelines(BaseTool):
    
    # set tool attributes
    name: str = 'Read Guidelines'  # name of the tool
    description: str = 'Reads organizational guidelines and returns the guideline content.'  # tool description
    result_as_answer: bool = True  # ensures the output is given as an answer

    # this method reads a text file containing the guidelines
    def _run(self, query: str) -> str:

        try:
            # define the remote file path
            file_path = 'https://raw.githubusercontent.com/GitiHubi/AgenticAuditing/master/guideline.txt'
            
            # open and read the content of the guideline file
            content = requests.get(file_path).text
            
            # return the file content as a string
            return str(content)
        
        except Exception as exception:
            
            # return error message if file reading fails
            return str(exception)

Das **ReadGuidelines**-Tool ermöglicht es dem **Senior Auditor**, auf das Richtliniendokument der Organisation zuzugreifen und es zu lesen. Dies unterstützt den Agenten dabei, die wichtigsten Punkte für Prüfungszwecke zusammenzufassen. Lassen Sie uns das Tool instanziieren:

In [ ]:
# instantiate the ReadGuidelines tool
rga = ReadGuidelines()

Mit dem eingerichteten `ReadGuidelines`-Tool definieren wir nun den **Senior Auditor**-Agenten. Dieser Agent liest organisatorische Richtlinien und fasst die wichtigsten Informationen für Prüfungszwecke zusammen. Das "Gehirn" des Agenten nutzt das **LLaMA 3.2-Modell**, das ihm fortschrittliche Sprachverstehens- und Zusammenfassungsfähigkeiten verleiht.

Der **Senior Auditor**-Agent besitzt die folgenden Eigenschaften:
- **Rolle:** Senior Auditor.  
- **Ziel:** Wichtige Informationen aus organisatorischen Richtlinien extrahieren und zusammenfassen.  
- **Hintergrundgeschichte:** Bekannt dafür, komplexe Richtlinien in umsetzbare Erkenntnisse zu übersetzen.

In [ ]:
# define the Senior Auditor agent, using LLaMA 3.2 as its language model
senior_auditor = Agent(
    role='Senior Auditor',  # the agent's role
    goal="Analyze comprehensive organizational guidelines and extract their main points.",  # the agent's task focus
    
    # detailed backstory providing context for the agent's role
    backstory="""You are a skilled Senior Auditor, known for your expertise in analyzing complex guidelines 
    and translating them into concise, actionable insights. Your summaries are critical for guiding 
    other audit team members in compliance and audit assessments.""",
    
    verbose=True,  # enables detailed logging for transparency during execution
    allow_delegation=False,  # disables task delegation for this agent to maintain focused task handling
    llm=llm_gpt,  # specifies the LLaMA 3.2 model as the agent's "brain" for language-based tasks
    tools=[rga]  # attaches the ReadGuidelines tool to enable reading and processing guideline documents
)

Der **Senior Auditor**-Agent wird mit der Analyse und Zusammenfassung der Richtlinien der Organisation beauftragt. Ziel ist es, eine prägnante, zugängliche Zusammenfassung zu erstellen, die sich insbesondere auf relevante Filterkriterien für die Prüfung von Zahlungen konzentriert.

Zusammenfassend soll die Aufgabe folgende Kriterien erfüllen:
- **Kriterium 1:** Klar und prägnant sein, mit minimalem Fachjargon.  
- **Kriterium 2:** Fokussierung auf Filterkriterien, die für die Prüfung von Zahlungen relevant sind.  
- **Kriterium 3:** Die Filterkriterien, die der IT-Prüfer verwenden sollte, explizit angeben.  
- **Kriterium 4:** Die Zusammenfassung sollte in nur zwei oder drei Sätzen erfolgen.

In [ ]:
# define the task for the Senior Auditor agent
analyze_guideline = Task(
    description=dedent("""\
        Summarize the given guideline and extract its main gist in a very concise and short manner for an IT auditor.

        Utilize the provided guideline text and insights to complete your task effectively.

        Your summary should be:
        1. Clear and concise, avoiding complex jargon to ensure readability.
        2. Focused on the key points relevant to deriving a SQL query for auditing payments.
        3. Explicitly state the filtering criteria that the IT auditor should use when analyzing the payments to identify potential violations of the guideline.
        4. Short, encompassing a total of two to three sentences (this is very important).
    """),
    expected_output=dedent("""\
        A concise and clear summary of the guideline, tailored for an IT auditor, including explicit filtering criteria for analyzing payments.
    """),
    agent=senior_auditor,  # assign the task to the senior auditor agent
    max_attempts=1,        # set the number of attempts allowed for this task
    output_file='task_01.txt'  # file to store the task output
)

Mit der nun definierten Aufgabe können wir sie ausführen. Der **Senior Auditor**-Agent wird die Richtlinien verarbeiten und eine Zusammenfassung entsprechend den angegebenen Kriterien erstellen.

In [ ]:
analyze_guideline.execute_sync();

### 3.2 Einrichtung eines Teams von Prüfungsagenten

In diesem Abschnitt erweitern wir das Prüfungsteam um zwei zusätzliche Agenten: den **Senior IT Auditor** und den **Audit Manager**. Jeder dieser Agenten bringt spezialisierte Fähigkeiten in den Prüfungsprozess ein und ermöglicht so einen umfassenden Workflow für die Prüfung von Buchhaltungsbuchungen. Der **Senior IT Auditor** wird sich auf die Analyse von Zahlungsdatensätzen in der Datenbank konzentrieren und die Einhaltung organisatorischer Richtlinien sicherstellen. Der **Audit Manager** hingegen wird die Ergebnisse in einem strukturierten Bericht zusammenfassen und so auch für ein nicht-technisches Publikum zugänglich machen.

#### 3.2.1 Einrichtung des Senior IT Auditor Agenten

Der zweite Agent, der **Senior IT Auditor**, ist auf das Abfragen von Datenbanken spezialisiert, um potenzielle Verstöße gegen Richtlinien aufzudecken. Das "Gehirn" dieses Agenten wird ebenfalls vom **LLaMA 3.2-Modell** betrieben, wodurch er sprachbasierte Aufgaben wie die Formulierung von Abfragen und die Datenanalyse durchführen kann (siehe Abbildung 3). Seine Hauptaufgabe besteht darin, Prüfungsrichtlinien in **SQL-Abfragen** zu übersetzen, Buchhaltungsbuchungen zu analysieren und hochpreisige Transaktionen zu identifizieren, die auf Compliance-Probleme hinweisen könnten.

Lassen Sie uns die Schritte zur Erstellung und Aufgabenvergabe an diesen Agenten durchgehen.

<img align='center' style="max-width: 850px" src='./senior_it_auditor_agent.png'>

**Figure 3: Senior IT Auditor Agent - Tools and Capabilities**

Bevor wir den **Senior IT Auditor** Agenten definieren, müssen wir eine in-memory SQLite-Datenbank einrichten, die Beispieldatensätze zu Zahlungen enthält. Dadurch kann der Agent SQL-Abfragen auf diese Datensätze ausführen, um mögliche Verstöße gegen die organisatorischen Richtlinien zu überprüfen.

In [ ]:
# define the metadata object
metadata_obj = MetaData()

# Define the schema for a payments table
payments = Table(
    "payments",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("text", String(4), nullable=False),
    Column("amount", Float, nullable=False),
    Column("date", Date, nullable=False),
)

# insert sample data for payments
payment_data = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 1029, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 1225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 1798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

# create the in-memory SQLite engine and apply the schema
engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

# insert data into the payments table
with engine.begin() as conn:
    for payment in payment_data:
        stmt = insert(payments).values(
            id=payment[0],
            text=payment[1],
            amount=payment[2],
            date=payment[3]
        )
        conn.execute(stmt)

Nachdem die in-memory Zahlungen-Datenbank eingerichtet wurde, definieren wir eine Funktion, die dem **Senior IT Auditor** ermöglicht, SQL-Abfragen auf der Datenbank auszuführen. Diese Funktion wird die Abfragen ausführen und die Ergebnisse in einem strukturierten Format zurückgeben.

In [ ]:
# define a function to execute a SQL query on the database
def run_query(query):
    try:
        # execute the query and fetch results
        result = pd.read_sql(query, con=engine)
        
        if result.empty:
            return "No results found."
        
        # format and return the result as a table
        table = tb.tabulate(result, headers=result.columns, tablefmt="grid")
        
        return table
    
    except Exception as e:
        return str(e)

Der **Senior IT Auditor** wird das Tool `QueryDatabase` verwenden, um SQL-Abfragen auf den Zahlungsdatensätzen auszuführen. Dieses Tool nimmt eine Abfrage als Eingabe, führt sie auf der Datenbank aus und gibt die Ergebnisse zurück.

In [ ]:
# define the QueryDatabase tool
class QueryDatabase(BaseTool):

    name: str = "Query database"
    description: str = "Execute SQL database query and return the query result."
    result_as_answer: bool = True

    # define database query
    def _run(self, query: str) -> str:

        try:
            query = query.strip().replace('"', '')

            # echoing cleaned query
            print(f"\nExecuting cleaned query: {query}")

            if not query:
                return "No SQL query provided."

            result = run_query(query)

            return result

        except Exception as exception:
            return str(exception)

Das Tool **QueryDatabase** ermöglicht es dem **Senior IT Auditor**, SQL-Abfragen auf der Zahlungen-Datenbank auszuführen, wodurch eine effiziente Prüfung von hochpreisigen Zahlungen anhand der angegebenen Filterkriterien ermöglicht wird. Lassen Sie uns das Tool instanziieren:

In [ ]:
# instantiate the QueryDatabase tool
qdb = QueryDatabase()

Als Nächstes definieren wir den **Senior IT Auditor** Agenten. Dieser Agent ist verantwortlich für die Umwandlung der organisatorischen Richtlinienzusammenfassung in SQL-Abfragen, um potenzielle Verstöße gegen Richtlinien in der Zahlungsdatenbank zu identifizieren. Der Agent wird das Tool `QueryDatabase` verwenden, um diese Abfragen auszuführen.

Der **Senior IT Auditor** Agent besitzt die folgenden Eigenschaften:
- **Rolle:** Senior Auditor.  
- **Ziel:** Richtlinienzusammenfassungen in SQL-Abfragen für die Prüfung von Zahlungsdatensätzen umwandeln.  
- **Hintergrundgeschichte:** Ein hochqualifizierter IT-Prüfer, bekannt für die Analyse komplexer Systeme und die Sicherstellung der Compliance.

In [ ]:
# define the Senior IT Auditor agent with a focus on querying payment records for potential compliance issues
senior_it_auditor = Agent(
    role='Senior IT Auditor',  # role designation for the agent
    goal=dedent("""\
        convert an organizational guidelines summary into a SQL query to determine guideline violations in the database of payment records.
    """),  # goal statement to translate audit requirements into actionable queries
    
    # detailed backstory highlighting the agent's role and expertise
    backstory=dedent("""\
        you are a highly skilled IT auditor at a leading financial auditing firm. 
        your expertise includes identifying system vulnerabilities, ensuring regulatory compliance, and optimizing payment processes.
        you excel in analyzing complex IT systems and providing actionable recommendations to enhance security and efficiency.
    """),
    
    verbose=True,  # enable detailed logging to monitor task execution
    allow_delegation=False,  # prevents task delegation to ensure focused task handling by this agent
    llm=llm_gpt,  # the agent's "brain," utilizing the LLaMA3 model for handling language-based tasks
    tools=[qdb]  # attach the QueryDatabase tool for SQL query execution on payment records
)

Der **Senior IT Auditor** Agent wird mit der Aufgabe betraut, die vom **Senior Auditor** bereitgestellten Filterkriterien in eine SQL-Abfrage zu übersetzen, die auf der Zahlungsdatenbank ausgeführt wird, um potenzielle Richtlinienverstöße, insbesondere hochpreisige Transaktionen, zu identifizieren.

Zusammenfassend soll die Aufgabe folgende Kriterien erfüllen:
- **Kriterium 1:** Eine klare SQL-Abfrage basierend auf den Filterkriterien des Senior Auditors ableiten.  
- **Kriterium 2:** Mit `*` alle Felder im Abfrageergebnis abrufen.  
- **Kriterium 3:** Abfrageergebnisse in einem strukturierten, lesbaren Format zurückgeben.

In [ ]:
# define the Query Database task for the Senior IT Auditor agent
query_database = Task(
    description=dedent("""\
        Translate the filtering criteria provided by the **Senior Auditor** into a SQL query and execute it to audit the relevant database for payments exceeding a specific threshold.
        
        Your actions should include:
        
        1. Use the explicit filtering criteria from the **Senior Auditor** to formulate a SQL query.
        2. Ensure that the query retrieves all relevant fields by using `*` instead of specifying individual field names.
        3. Execute the query and format the results in a structured format, such as JSON or a well-formatted table.
        
        The input should always adhere to this format:
        
        {"query": "SELECT * FROM payments WHERE <filtering_criteria>"}

    """),
    expected_output=dedent("""\
        A table containing all fields (id, text, amount, date) for payments that meet the filtering criteria provided by the Senior Auditor.
    """),  # expected output in a clear and structured format
    
    agent=senior_it_auditor,  # assign the task to the Senior IT Auditor agent
    max_attempts=1,           # limit task attempts to one to maintain workflow control
    output_file='task_02.txt' # specify output file to store the task results
)

#### 3.2.2 Einrichtung des Audit Manager Agenten

Der dritte Agent, der **Audit Manager**, fasst die Ergebnisse des Prüfungsteams in einem klaren und zugänglichen Bericht zusammen. Dieser Agent ist darauf spezialisiert, komplexe Prüfdaten in prägnante Zusammenfassungen zu destillieren, um Stakeholdern die wichtigsten Erkenntnisse ohne technischen Fachjargon zu vermitteln. Der **Audit Manager** benötigt keine externen Tools; er nutzt das **LLaMA 3.2-Modell**, um strukturierte und umsetzbare Berichte zu erstellen (siehe Abbildung 4).

Lassen Sie uns die Schritte zur Erstellung und Aufgabenvergabe an diesen Agenten durchgehen.

<img align='center' style='max-width: 900px' src='./audit_manager_agent.png'>

**Figure 4: Senior Audit Manager Agent - Tools and Capabilities**

Nun definieren wir den **Audit Manager** Agenten. Dieser Agent hat die Aufgabe, die Ergebnisse in einen klaren und umfassenden Prüfungsbericht zu synthetisieren, der den Prüfungsprozess, die Ergebnisse und umsetzbare Empfehlungen effektiv kommuniziert. Basierend auf den Erkenntnissen des **Senior IT Auditor** liefert der **Audit Manager** wirkungsvolle Berichte, die für ein breites Publikum leicht verständlich sind.

Der **Audit Manager** Agent besitzt die folgenden Eigenschaften:
- **Rolle:** Audit Manager.  
- **Ziel:** Prüfungsergebnisse in einen aufschlussreichen, zugänglichen und umsetzbaren Bericht zusammenfassen.  
- **Hintergrundgeschichte:** Ein gefeierter Prüfer, der komplexe Prüfdaten in klare Erkenntnisse umwandelt.

In [ ]:
audit_manager = Agent(
    role='Manager Auditor',
    goal=dedent("""\
        Create comprehensive and compelling audit reports that provide clear insights and actionable recommendations.
    """),
    backstory=dedent("""\
        You are a renowned Audit Manager, celebrated for your ability to produce insightful and impactful audit reports.
        Your reports distill complex audit findings into clear, accessible, and actionable insights, making them valuable to stakeholders.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm_gpt, # the agent will use the LLaMA3 model
    tools=[]  # the Audit Manager doesn't need any tools here
)

Der **Audit Manager** Agent wird damit beauftragt, die Prüfungsergebnisse des **Senior IT Auditor** in einem umfassenden Bericht zusammenzustellen. Der Bericht soll den Stakeholdern einen strukturierten Überblick über den Prüfungsprozess, Einblicke in die Ergebnisse und umsetzbare Empfehlungen auf Grundlage der Prüfergebnisse liefern.

Zusammenfassend soll die Aufgabe folgende Kriterien erfüllen:
- **Kriterium 1:** Klar und zugänglich für ein nicht-technisches Publikum sein.  
- **Kriterium 2:** Die Prüfungsverfahren detailliert beschreiben, einschließlich des Prüfungsziels und der verwendeten Methoden.  
- **Kriterium 3:** Die wichtigsten Ergebnisse prägnant präsentieren und auf identifizierte Probleme hinweisen.  
- **Kriterium 4:** Stakeholdern umsetzbare Empfehlungen auf Grundlage der Prüfergebnisse anbieten.

In [ ]:
# define the report writing task for the Audit Manager
write_report = Task(
    description=dedent("""\
        Compile a comprehensive audit report using the findings from the Senior IT Auditor. This report should clearly communicate the audit's process, results, and recommendations to stakeholders.

        In summary, your report should comply with the following criteria:
        
        1. Be clear and accessible for a non-technical audience.
        2. Include a detailed overview of the audit procedures, providing context and explaining the audit objectives.
        3. Present key findings concisely, highlighting significant observations such as high-value transactions.
        4. Offer actionable recommendations for stakeholders based on the findings.
        
        Your report structure should include:

        1. **Introduction**: Brief context and purpose of the audit.
        2. **Audit Procedures**: Description of methods used and rationale.
        3. **Findings**: Summary of query results, particularly high-value transactions or anomalies.
        4. **Recommendations**: Practical suggestions addressing any issues uncovered.
        5. **Conclusion**: Summary of key insights and implications for future audits.

        The goal is to create a report that effectively communicates the audit findings and provides valuable insights to stakeholders.
        
        As the report writing expert, you are responsible for producing this report without requesting additional information. 
        Utilize the provided data and insights to complete your task effectively.
    """),
    expected_output=dedent("""\
        A full audit report presented in a clear and accessible manner.
    """),
    agent=audit_manager,  # assign the task to the audit manager agent
    max_attempts=1,
    output_file='task_03.txt'
)

Mit den nun definierten Agenten können wir das Prüfungsteam etablieren. Die Agenten werden die Prüfung von Buchhaltungsbuchungen im Rahmen der im nächsten Abschnitt umgesetzten Verfahren durchführen.

## 4. Ausführen des Agentic Journal Entry Testings

In diesem abschliessenden Abschnitt bringen wir alle erstellten Agenten – den **Senior Auditor**, den **Senior IT Auditor** und den **Audit Manager** – zusammen und weisen ihnen ihre jeweiligen Aufgaben zu. Gemeinsam bilden sie das **Audit Team**, das den gesamten Prüfungsprozess in Zusammenarbeit durchführen wird.

Das **Audit Team** arbeitet sequentiell:
1. Der **Senior Auditor** analysiert die Richtlinien und extrahiert die wichtigsten Punkte.
2. Der **Senior IT Auditor** übersetzt diese Punkte in SQL-Abfragen und führt sie in der Zahlungsdatenbank aus.
3. Der **Audit Manager** fasst die Ergebnisse in einem umfassenden Prüfungsbericht zusammen.

Lassen wir das Team an die Arbeit gehen!

In [ ]:
# instantiate the audit team
audit_team = Crew(
    agents=[senior_auditor, senior_it_auditor, audit_manager],  # assign agents to the audit team
    tasks=[analyze_guideline, query_database, write_report],     # assign tasks to the audit team
    verbose=False,  # set logging level for output visibility
    process=Process.sequential  # set the process to sequential to ensure proper order of execution
)

Nachdem das **Audit Team** instanziiert wurde, können wir das Team laufen lassen, um die Prüfung durchzuführen. Die Agenten bearbeiten ihre Aufgaben nacheinander und stellen sicher, dass jeder Schritt in der richtigen Reihenfolge abgeschlossen wird.

In [ ]:
# get the audit team to work
result = audit_team.kickoff();

## Lab Zusammenfassung:

In diesem Lab haben Sie die folgenden zentralen Erkenntnisse gewonnen:

> 1. **Verständnis von Agentic Auditing:** Sie haben die Grundprinzipien des agentenbasierten Auditings gelernt und erfahren, wie KI-Agenten strukturierte Prüfungsaufgaben, insbesondere die Prüfung von Buchhaltungsbuchungen, automatisieren.  
> 2. **Implementierung von Agenten mit CrewAI:** Sie haben praktische Erfahrung im Einrichten von Agenten in CrewAI gesammelt, einschließlich eines **Senior Auditor** zur Zusammenfassung von Richtlinien, eines **Senior IT Auditor** für Datenbankabfragen und eines **Audit Manager** zur Berichtserstellung.  
> 3. **Erstellung agentenbasierter Workflows für die Prüfung von Buchhaltungsbuchungen:** Sie haben einen sequentiellen Workflow erstellt, um Richtlinien zu analysieren, Finanzdatensätze abzufragen und die Ergebnisse in einem vollständigen Prüfungsbericht zusammenzufassen.

Dieses Lab bot Einblicke in die Konzeption und Implementierung von KI-Agenten für Aufgaben der Finanzprüfung. Durch Einrichtung, Aufgabenkonfiguration und Zusammenarbeit der Agenten haben Sie Fähigkeiten zur Nutzung agentenbasierter Workflows zur Verbesserung von Prüfungsprozessen entwickelt.